In [ ]:
import pandas as pd
import os

In [ ]:
path = ""
donor_blood_group_file = ""
txm_event_name = ""
# starting_pattern = "W-"
starting_pattern = "KPPD_"

In [ ]:
donor_blood_groups = (pd.read_csv(os.path.join(path, donor_blood_group_file), delimiter=";")
.set_index("FamilyID").BloodType)

In [ ]:
infos = []
for i in os.listdir(path):
    if i.startswith(starting_pattern) and i.endswith(".csv"):
        df = pd.read_csv(os.path.join(path, i), delimiter=";")


        new = df[["FamilyID","BloodType","PatientTyping","DonorID","DonorTyping", "Gender", "DOB"]].iloc[0]
        if new.FamilyID == 'W-77R':
            print("special")
            luminex_col = 'NormalValue'
        else:
            luminex_col = "RawData"
        tt = (df[["Specificity",luminex_col]]
         .assign(hla_code = lambda df: df.Specificity.str.split(",").apply(lambda x: [y for y in x if y!="-"]))
         .loc[lambda df: df.hla_code.apply(len)>0]
        )

        antibodies=[]
        for _, row in tt.iterrows():
            if len(row.hla_code) == 1:
                antibodies.append((row.hla_code[0], int(float(row[luminex_col].replace(",",".")))))
            elif len(row.hla_code) == 2:
                hlas = row.hla_code
                start_code = hlas[0][0:2]
                if start_code in {"DP", "DQ"}:
                    first_number_code = hlas[0].split("*")[-1]
                    second_number_code = hlas[1].split("*")[-1]
                    antibodies.append((
                        f"{start_code}[{first_number_code},{second_number_code}]",
                        int(float(row[luminex_col].replace(",",".")))
                    ))
                else:
                    raise ValueError(f"Unexpected start of hla code that has multiple values per row: {hlas[0][0:2]}")
            else:
                raise ValueError(f"Unexpected number of hla codes on one row: {len(row.hla_code)}")


        new["RecipientAntibodies"] = antibodies
        new["DonorBloodType"] = donor_blood_groups.loc[new.DonorID]

        infos.append(new.to_frame().T)
        dum = df
df = pd.concat(infos)

In [ ]:
dff = df.rename(columns = {"FamilyID": "RecipientID", "PatientTyping": "RecipientTyping"})

In [ ]:
dff.iloc[0,7]

In [ ]:
donors = [
    {
        'medical_id': row.DonorID,
        'blood_group': row.DonorBloodType,
        'hla_typing': row.DonorTyping.split(","),
        'donor_type': "DONOR",

        'related_recipient_medical_id': row.RecipientID
    }
    for _, row in dff.iterrows()
]

acg_for_zero = ["A","B", "O"]
acg_for_other = ["A","B", "O", "AB"]
recipients = [
    {
        'medical_id': row.RecipientID,
        'blood_group': row.BloodType,
        'hla_typing': row.RecipientTyping.split(","),
        'year_of_birth': int(row.DOB.split(".")[-1]),
        'sex': row.Gender if not pd.isna(row.Gender) else None,
        'acceptable_blood_groups': acg_for_zero if row.BloodType == "O" else acg_for_other,
        "hla_antibodies": [
            {
                "mfi": antibody[1],
                "name": antibody[0],
                "cutoff": 2000
            }
            for antibody in row.RecipientAntibodies
        ]
    }
    for _, row in dff.iterrows()
]




input_json = {
    "country": "AUT",
    "txm_event_name": txm_event_name,
    "donors": donors,
    "recipients": recipients
}

In [ ]:
import json
with open(os.path.join(path, "wien_export.json"), 'w') as f:

    json.dump(input_json, f)